### **README**

# What this does

This script allows you to download a **dashboard** into a singular Excel file. Each **Tile** will be a tab/worksheet within that Excel file. Each tab's title will be 'Sheet `<i of tile>`. You can also specify a file name for the download.

# Notes
`download_multitile_tabbed(dash_id, file_name)` accepts two inputs:

- `dash_id`(string) - accepts a dashboard ID. 
- `file_name`(string) - the desired output file name of the final excel file.

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/


In [1]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 317kB 5.8MB/s 


In [137]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv
import json
import re

In [138]:
## inputs ##

dash_id = '1317'
file_name = 'Tabbed_Dashboard'


In [141]:
def download_multitile_tabbed(dash_id, file_name):
# variables 
  data = sdk.search_dashboard_elements(dashboard_id=dash_id)
  listlen = len(data)
  query_ids = []

# helper methods
  def id_getter(data, listlen, query_ids):
    for i in range(listlen):
      query_ids.append(data[i].result_maker.query_id)
    return query_ids
  
  def downloader(data, listlen, query_ids, file_name):
    idlist = id_getter(data, listlen, query_ids)
    with pd.ExcelWriter(file_name+'.xlsx') as writer:
      for i in range(listlen):
        resultset = sdk.run_query(query_id=idlist[i],result_format='csv')
        with open('temp.csv', 'w', newline='') as file:
          file.write(resultset)
        df = pd.read_table('temp.csv', delimiter =",",engine='python')
        df.to_excel(writer, sheet_name=('Sheet ' + str(i)))
# call downloader
  downloader(data, listlen, query_ids, file_name)

In [140]:
# test it yourself
download_multitile_tabbed(dash_id, file_name)

# colab only
from google.colab import files 
files.download(file_name+'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>